### Шаг 1: Первичная обработка данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_prolongations = pd.read_csv('prolongations.csv')
df_financial = pd.read_csv('financial_data.csv', decimal=',', thousands=' ')

In [3]:
df_prolongations.head()

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


In [4]:
df_financial.head(3)

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


Определяем колонки, которые не являются месяцами

In [5]:
id_vars = ['id', 'Причина дубля', 'Account']

Определяем колонки, которые являются месяцами и содержат суммы отгрузок

In [6]:
value_vars = [col for col in df_financial.columns if col not in id_vars]

Преобразуем ("расплавляем") "широкую" таблицу в "длинную"

In [7]:
df_financial_long = pd.melt(
    df_financial,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='shipment_month_str', # Новая колонка для названий месяцев
    value_name='shipment_amount'   # Новая колонка для сумм отгрузок
)

Обрабатываем специальные текстовые значения в суммах отгрузок

Сначала заменяем "в ноль" на специальный маркер, например -1, чтобы отличить от настоящих нулей

In [8]:
df_financial_long['shipment_amount'] = df_financial_long['shipment_amount'].replace('в ноль', -1)

Заменяем 'стоп' и 'end' на NaN (Not a Number), чтобы их можно было легко исключить из математических операций. Они будут обозначать отсутствие данных.

In [9]:
df_financial_long['shipment_amount'] = df_financial_long['shipment_amount'].replace(['стоп', 'end'], np.nan)

Находим все project_id, у которых есть хотя бы одно значение NaN (бывший 'стоп'/'end')

In [10]:
stopped_projects = df_financial_long[df_financial_long['shipment_amount'].isna()]['id'].unique()

Удаляем ВСЕ данные по этим проектам из таблицы

In [11]:
df_financial_long = df_financial_long[~df_financial_long['id'].isin(stopped_projects)]

Функция для преобразования числовых значений

In [12]:
def clean_value(value):
    if isinstance(value, str):
        value = value.replace('\xa0', '').replace(' ', '').replace(',', '.')
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0.0

Применяем

In [13]:
df_financial_long['shipment_amount'] = df_financial_long['shipment_amount'].apply(clean_value)

Преобразуем текстовые даты в datetime объекты

In [14]:
month_map = {'Январь':'01','Февраль':'02','Март':'03','Апрель':'04','Май':'05','Июнь':'06','Июль':'07','Август':'08','Сентябрь':'09','Октябрь':'10','Ноябрь':'11','Декабрь':'12'}
date_series = df_financial_long['shipment_month_str'].str.split(' ')
df_financial_long['Дата_norm'] = '01-' + date_series.str[0].map(month_map) + '-' + date_series.str[1]
df_financial_long['shipment_month_str'] = pd.to_datetime(df_financial_long['Дата_norm'], format='%d-%m-%Y')

То же самое проделываем для таблицы пролонгаций

In [15]:
date_series = df_prolongations['month'].str.split(' ')
df_prolongations['Дата_norm'] = '01-' + date_series.str[0].str.capitalize().map(month_map) + '-' + date_series.str[1]
df_prolongations['month'] = pd.to_datetime(df_prolongations['Дата_norm'], format='%d-%m-%Y')

In [16]:
df_prolongations = df_prolongations.drop(columns=['Дата_norm'])
df_financial_long = df_financial_long.drop(columns=['Дата_norm'])

In [17]:
df_prolongations.head()

,id,month,AM
0,42,2022-11-01,Васильев Артем Александрович
1,453,2022-11-01,Васильев Артем Александрович
2,548,2022-11-01,Михайлов Андрей Сергеевич
3,87,2022-11-01,Соколова Анастасия Викторовна
4,429,2022-11-01,Соколова Анастасия Викторовна


In [18]:
df_financial_long.head()

,id,Причина дубля,Account,shipment_month_str,shipment_amount
211,268,NaN,Смирнова Ольга Владимировна,2022-11-01,79235.0
235,676,первая часть оплаты,Смирнова Ольга Владимировна,2022-11-01,388195.0
236,676,вторая часть оплаты,Смирнова Ольга Владимировна,2022-11-01,14280.0
240,668,первая часть оплаты,Попова Екатерина Николаевна,2022-11-01,27950.0
241,668,вторая часть оплаты,Попова Екатерина Николаевна,2022-11-01,24401.0


Сортируем данные, чтобы месяцы шли по порядку для каждого проекта

In [19]:
df_financial_long = df_financial_long.sort_values(by=['id', 'shipment_month_str'])

Для каждого проекта ('id') заполняем значения -1 предыдущим значением ffill - forward fill

In [20]:
df_financial_long['shipment_amount'] = df_financial_long.groupby('id')['shipment_amount'].transform(
    lambda x: x.replace(-1, method='ffill')
)

C:\Users\scarl\AppData\Local\Temp\ipykernel_17504\1522537242.py:2: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  lambda x: x.replace(-1, method='ffill')


Если после этого все еще остались -1 (например, в самом первом месяце проекта), то их уже можно заменить на 0

In [21]:
df_financial_long['shipment_amount'] = df_financial_long['shipment_amount'].replace(-1, 0)

In [22]:
df_merged = pd.merge(
    df_prolongations,
    df_financial_long,
    on='id',
    how='inner'
)

In [23]:
df_merged.head()

,id,month,AM,Причина дубля,Account,shipment_month_str,shipment_amount
0,547,2022-11-01,Михайлов Андрей Сергеевич,основные работы,Соколова Анастасия Викторовна,2022-11-01,144180.0
1,547,2022-11-01,Михайлов Андрей Сергеевич,основные работы,Соколова Анастасия Викторовна,2022-12-01,104660.0
2,547,2022-11-01,Михайлов Андрей Сергеевич,основные работы,Соколова Анастасия Викторовна,2023-01-01,226420.0
3,547,2022-11-01,Михайлов Андрей Сергеевич,основные работы,Соколова Анастасия Викторовна,2023-02-01,139660.0
4,547,2022-11-01,Михайлов Андрей Сергеевич,основные работы,Соколова Анастасия Викторовна,2023-03-01,165540.0


In [24]:
df_clean = df_merged.drop(columns=['Account', 'Причина дубля'])

In [25]:
df_agg = df_clean.groupby(['id', 'AM', 'shipment_month_str', 'month'])['shipment_amount'].sum().reset_index()

In [26]:
df_cleanest = df_agg.rename(columns={
    'id': 'project_id',
    'AM': 'account_manager',
    'month': 'project_end_month', # Последний месяц реализации проекта
    'shipment_month_str': 'shipment_month',       # Месяц, за который произведена отгрузка
    'shipment_amount': 'shipment_sum'        # Сумма отгрузки
})

In [27]:
df_cleanest

,project_id,account_manager,shipment_month,project_end_month,shipment_sum
0,190,Васильев Артем Александрович,2022-11-01,2023-02-01,149080.0
1,190,Васильев Артем Александрович,2022-11-01,2023-12-01,149080.0
2,190,Васильев Артем Александрович,2022-12-01,2023-02-01,149100.0
3,190,Васильев Артем Александрович,2022-12-01,2023-12-01,149100.0
4,190,Васильев Артем Александрович,2023-01-01,2023-02-01,177595.0
...,...,...,...,...,...
299,707,Соколова Анастасия Викторовна,2023-10-01,2023-10-01,65700.0
300,707,Соколова Анастасия Викторовна,2023-11-01,2023-10-01,69190.0
301,707,Соколова Анастасия Викторовна,2023-12-01,2023-10-01,70230.0
302,707,Соколова Анастасия Викторовна,2024-01-01,2023-10-01,71710.0


### Шаг 3: Расчет коэффициентов пролонгации

Используем DateOffset для удобной работы с датами (например, "минус один месяц")

In [28]:
from pandas.tseries.offsets import DateOffset

In [29]:
managers = df_cleanest['account_manager'].unique().tolist()
all_managers_label = 'Весь отдел'
managers_and_total = managers + [all_managers_label]

Создаем список дат для каждого месяца 2023 года, по которому будем строить отчет

In [30]:
report_months = pd.to_datetime([f'2023-{i}-01' for i in range(1, 13)])

Инициализируем пустой список, куда будем складывать результаты расчетов

In [31]:
results = []

Основной цикл:

In [32]:
# Итерируемся по каждому менеджеру (и по всему отделу)
for manager in managers_and_total:
    
    # Если текущее имя - 'Весь отдел', используем весь датафрейм. 
    # Иначе - фильтруем данные по конкретному менеджеру.
    if manager == all_managers_label:
        manager_df = df_cleanest
    else:
        manager_df = df_cleanest[df_cleanest['account_manager'] == manager]

    # Инициализируем переменные для подсчета годовых итогов
    total_k1_numerator, total_k1_denominator = 0, 0
    total_k2_numerator, total_k2_denominator = 0, 0

    # Теперь итерируемся по каждому месяцу 2023 года
    for current_month in report_months:

        # Расчет Коэффициента пролонгации на 1-й месяц
        prev_month = current_month - DateOffset(months=1)

        # ЗНАМЕНАТЕЛЬ: Находим проекты, завершившиеся в прошлом месяце, и сумму их отгрузки
        # за тот же (прошлый) месяц.
        k1_base_df = manager_df[
            (manager_df['project_end_month'] == prev_month) &
            (manager_df['shipment_month'] == prev_month)
        ]
        k1_denominator = k1_base_df['shipment_sum'].sum()

        # ЧИСЛИТЕЛЬ: Для этих же проектов смотрим их отгрузку в ТЕКУЩЕМ месяце.
        k1_prolonged_project_ids = k1_base_df['project_id'].unique()
        k1_numerator = manager_df[
            (manager_df['project_id'].isin(k1_prolonged_project_ids)) &
            (manager_df['shipment_month'] == current_month)
        ]['shipment_sum'].sum()

        # Расчет Коэффициента пролонгации на 2-й месяц
        month_minus_2 = current_month - DateOffset(months=2)

        # ЗНАМЕНАТЕЛЬ:
        # 1. Находим ID проектов, завершившихся 2 месяца назад.
        ended_m2_ids = manager_df[manager_df['project_end_month'] == month_minus_2]['project_id'].unique()

        # 2. Находим ID проектов из п.1, у которых БЫЛА отгрузка в прошлом месяце (т.е. они были пролонгированы на 1-й месяц).
        prolonged_in_1st_month_ids = manager_df[
            (manager_df['project_id'].isin(ended_m2_ids)) &
            (manager_df['shipment_month'] == prev_month) &
            (manager_df['shipment_sum'] > 0)
        ]['project_id'].unique()

        # 3. Находим ID проектов для нашей базы - те, что завершились 2 месяца назад и НЕ были пролонгированы на 1-й месяц.
        k2_base_project_ids = np.setdiff1d(ended_m2_ids, prolonged_in_1st_month_ids)

        # 4. Считаем сумму отгрузки для этих "непролонгированных" проектов за их последний месяц (т.е. 2 месяца назад).
        k2_denominator = manager_df[
            (manager_df['project_id'].isin(k2_base_project_ids)) &
            (manager_df['shipment_month'] == month_minus_2)
        ]['shipment_sum'].sum()

        # ЧИСЛИТЕЛЬ: Для этих же "непролонгированных" проектов ищем отгрузку в ТЕКУЩЕМ месяце.
        k2_numerator = manager_df[
            (manager_df['project_id'].isin(k2_base_project_ids)) &
            (manager_df['shipment_month'] == current_month)
        ]['shipment_sum'].sum()

        # Рассчитываем коэффициенты, избегая деления на ноль
        k1 = k1_numerator / k1_denominator if k1_denominator > 0 else 0
        k2 = k2_numerator / k2_denominator if k2_denominator > 0 else 0

        # Сохраняем результат за месяц в наш список
        results.append({
            'Менеджер': manager,
            'Метрика': 'Коэффициент 1',
            'Месяц': current_month,
            'Значение': k1
        })
        results.append({
            'Менеджер': manager,
            'Метрика': 'Коэффициент 2',
            'Месяц': current_month,
            'Значение': k2
        })

        # Добавляем месячные суммы к годовым
        total_k1_numerator += k1_numerator
        total_k1_denominator += k1_denominator
        total_k2_numerator += k2_numerator
        total_k2_denominator += k2_denominator
    
    # Рассчитываем годовые коэффициенты
    annual_k1 = total_k1_numerator / total_k1_denominator if total_k1_denominator > 0 else 0
    annual_k2 = total_k2_numerator / total_k2_denominator if total_k2_denominator > 0 else 0

    # Сохраняем годовой результат
    results.append({
        'Менеджер': manager,
        'Метрика': 'Коэффициент 1 (Год)',
        'Месяц': pd.to_datetime('2023-12-31'), # для сортировки
        'Значение': annual_k1
    })
    results.append({
        'Менеджер': manager,
        'Метрика': 'Коэффициент 2 (Год)',
        'Месяц': pd.to_datetime('2023-12-31'), # для сортировки
        'Значение': annual_k2
    })

Создаем DataFrame из списка с результатами

In [33]:
results_df = pd.DataFrame(results)

In [34]:
results_df

,Менеджер,Метрика,Месяц,Значение
0,Васильев Артем Александрович,Коэффициент 1,2023-01-01,2.403191
1,Васильев Артем Александрович,Коэффициент 2,2023-01-01,0.000000
2,Васильев Артем Александрович,Коэффициент 1,2023-02-01,0.000000
3,Васильев Артем Александрович,Коэффициент 2,2023-02-01,0.000000
4,Васильев Артем Александрович,Коэффициент 1,2023-03-01,1.754292
...,...,...,...,...
177,Весь отдел,Коэффициент 2,2023-11-01,0.000000
178,Весь отдел,Коэффициент 1,2023-12-01,2.000000
179,Весь отдел,Коэффициент 2,2023-12-01,0.000000
180,Весь отдел,Коэффициент 1 (Год),2023-12-31,2.562930


Создаем сводную pivot-таблицу. Индекс - менеджеры, колонки - месяцы, значения - коэффициенты

In [35]:
report_pivot = results_df.pivot_table(
    index='Менеджер', 
    columns=['Месяц', 'Метрика'], 
    values='Значение'
)

Сохраняем в Excel

In [36]:
file_name = 'analytical_report.xlsx'
with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
    report_pivot.to_excel(writer, sheet_name='Отчет по пролонгациям', merge_cells=True)

    # Добавляем форматирование для наглядности
    workbook  = writer.book
    worksheet = writer.sheets['Отчет по пролонгациям']